In [1]:

from dotenv import load_dotenv
load_dotenv()
import os
import sys
sys.path.append(
    os.environ.get('WORK_DIR')) #type: ignore
sys.path.append(
    os.environ.get('DBASE_DIR')) #type: ignore
from dbase.DataAPI.ThetaData import * #type: ignore
from dbase.database.SQLHelpers import * #type: ignore
import pandas as pd
from EventDriven.data import HistoricTradeDataHandler
from EventDriven.event import *
from queue import Queue
from trade.backtester_.backtester_ import PTDataset, PTBacktester
import pandas_ta as ta
from trade.assets.Stock import Stock
from trade.backtester_.utils.WalkForwardUtils import prev_monday 
from trade.backtester_.strats import BBandsTrend2
from trade.backtester_.strats import MAStrat
import yfinance as yf
from datetime import datetime
from EventDriven.backtest import OptionSignalBacktest
from copy import deepcopy
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2


Console Logging & File Logging Can be configured using STREAM_LOG_LEVEL and FILE_LOG_LEVEL in environment variables.
Propagate to root logger can be set using PROPAGATE_TO_ROOT_LOGGER in environment variables.
Example:
STREAM_LOG_LEVEL = 'DEBUG'
FILE_LOG_LEVEL = 'INFO'
PROPAGATE_TO_ROOT_LOGGER = 'False'

2025-02-26 06:59:49 trade.helpers.Logging INFO: Logging Root Directory: /Users/chiemelienwanisobi/cloned_repos/QuantTools/logs
Using Proxy URL: http://18.232.166.224:5500/thetadata


In [2]:
import json
with open('/Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/input/profitable_weights.json', 'r') as f:
    weights = json.load(f)
ttrades__ = pd.read_csv('/Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/input/profitable_trades.csv').iloc[:, 1:]


In [3]:
trades_ = ttrades__.copy()
trades_

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,Ticker
0,34.0,504.0,524.0,279.795877,253.010232,-910.711955,-0.095733,2023-07-05,2023-08-02,28 days,TSLA
1,28.0,504.0,526.0,192.240502,185.493749,-188.909083,-0.035095,2023-07-05,2023-08-04,30 days,AAPL
2,17.0,504.0,529.0,336.262811,322.011093,-242.279208,-0.042383,2023-07-05,2023-08-09,35 days,MSFT
3,48.0,504.0,536.0,87.043588,82.000000,-242.092217,-0.057943,2023-07-05,2023-08-18,44 days,AVGO
4,11.0,504.0,583.0,130.695846,122.257034,-92.826927,-0.064568,2023-07-05,2023-10-25,112 days,AMZN
5,178.0,504.0,753.0,42.282471,123.470001,14451.380412,1.920123,2023-07-05,2024-07-01,362 days,NVDA
6,1.0,522.0,522.0,170.283917,168.355807,-1.928110,-0.011323,2023-07-31,2023-07-31,0 days,JNJ
7,5.0,522.0,556.0,332.810769,318.700396,-70.551863,-0.042398,2023-07-31,2023-09-18,49 days,HD
8,5.0,530.0,535.0,239.284572,224.549052,-73.677602,-0.061582,2023-08-10,2023-08-17,7 days,BA
9,5.0,530.0,535.0,53.784257,52.084613,-8.498219,-0.031601,2023-08-10,2023-08-17,7 days,WMT


In [4]:
symbol_list = trades_.Ticker.unique()
untraded_symbols = [s for s in weights.keys() if s not in trades_.Ticker.unique()]
for s in untraded_symbols:
    weights.pop(s)
weights

{'TSLA': 0.17420799855438768,
 'NVDA': 0.13696693517022676,
 'AAPL': 0.10023373641816916,
 'MSFT': 0.10467300141772679,
 'AVGO': 0.07698902053627323,
 'SBUX': 0.0790869263496488,
 'AMD': 0.030287217010091606,
 'GOOG': 0.06259616627482918,
 'BAC': 0.056805302639254644,
 'QCOM': 0.025190453887818636,
 'AMZN': 0.02757812121505515,
 'HD': 0.03573121193888573,
 'BA': 0.02198114951948728,
 'DIS': 0.024661026395236287,
 'MU': 0.010524714558475388,
 'WMT': 0.005,
 'JNJ': 0.005,
 'INTC': 0.005}

In [10]:

from EventDriven.riskmanager import RiskManager, close_cache, spot_cache, chain_cache, oi_cache, LOOKBACKS, order_cache
from pandas.tseries.offsets import BDay

rm = RiskManager(None, None, 1000000)
rm.OrderPicker.liquidity_threshold = 100
rm.OrderPicker.lookback = 10
rm.OrderPicker.data_availability_threshold = 0.5
date, tick = '2023-07-05', 'AVGO'
date, tick = '2023-12-01', 'BAC'
start = (pd.to_datetime(date) - BDay(30)).strftime('%Y-%m-%d')
right = 'C'
order_settings = {'type': 'naked',
 'specifics': [{'direction': 'long',
   'rel_strike': .800,
   'dte': 365,
   'moneyness_width': 0.35},
   {'direction': 'short',
  'rel_strike': .60,
  'dte': 365,
  'moneyness_width': 0.35}
],
 'name': 'vertical_spread'}



order =  rm.OrderPicker.get_order(tick, date, right, 2, order_settings)
order

{'result': 'SUCCESSFUL',
 'data': {'long': ['BAC20250117C35'],
  'short': ['BAC20250117C55'],
  'trade_id': '&L:BAC20250117C35&S:BAC20250117C55',
  'close': 1.9900000000000002}}

In [16]:
list(Stock.list_instances().values())[0].spot(spot_type = 'chain_price')

{Timestamp('2023-12-01 16:00:00'): 30.959999084472656}

In [30]:
for i, item in order_cache.items():
    for k, order in item.items():
        if order['result'] == 'SUCCESSFUL' and k == 'BAC':
            print(k,order)

BAC {'result': 'SUCCESSFUL', 'data': {'long': ['BAC20250117C35'], 'short': ['BAC20250117C55'], 'trade_id': '&L:BAC20250117C35&S:BAC20250117C55', 'close': 5.699999999999999}}


In [18]:
#Backtest class 
## Find a way to not always reinitialize the backtest class, when want to redo
evb_backtest = OptionSignalBacktest(trades_, initial_capital=20_000)

In [19]:
evb_backtest.portfolio.initial_capital

20000

In [21]:
w_map = {x: w  * 0.75 for x, w in weights.items()}
evb_backtest.portfolio.weight_map = w_map
evb_backtest.portfolio.weight_map
evb_backtest.portfolio.risk_manager.OrderPicker.liquidity_threshold = 100
evb_backtest.portfolio.risk_manager.OrderPicker.lookback = 10
evb_backtest.portfolio.risk_manager.OrderPicker.data_availability_threshold = 0.5
evb_backtest.portfolio.order_settings = {'type': 'naked',
 'specifics': [{'direction': 'long',
   'rel_strike': .800,
   'dte': 365,
   'moneyness_width': 0.35},
   {'direction': 'short',
  'rel_strike': .60,
  'dte': 365,
  'moneyness_width': 0.35}
],
 'name': 'vertical_spread'}

evb_backtest.portfolio.max_contract_price = 2
evb_backtest.executor.commission_rate = 0.65
evb_backtest.executor.commission_rate
evb_backtest.portfolio.allocated_cash_map

{'TSLA': 2613.1199783158154,
 'AAPL': 1503.5060462725376,
 'MSFT': 1570.0950212659018,
 'AVGO': 1154.8353080440984,
 'AMZN': 413.6718182258273,
 'NVDA': 2054.5040275534016,
 'JNJ': 75.0,
 'HD': 535.968179083286,
 'BA': 329.71724279230926,
 'WMT': 75.0,
 'INTC': 75.0,
 'QCOM': 377.8568083172795,
 'SBUX': 1186.3038952447318,
 'AMD': 454.3082551513741,
 'MU': 157.87071837713083,
 'DIS': 369.9153959285443,
 'BAC': 852.0795395888197,
 'GOOG': 938.9424941224378}

In [22]:
signals = evb_backtest.bars.signal_df
signals

,Date,TSLA,AAPL,MSFT,AVGO,AMZN,NVDA,JNJ,HD,BA,WMT,INTC,QCOM,SBUX,AMD,MU,DIS,BAC,GOOG
0,2023-07-05,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0
1,2023-07-06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2023-07-07,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2023-07-08,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2023-07-09,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,2024-06-27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
359,2024-06-28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
360,2024-06-29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
361,2024-06-30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
signals_df = deepcopy(signals).set_index('Date')
signals_df[signals_df!=-1].sum().sum()

31.0

In [24]:
import cProfile
import pstats
import io

profiler = cProfile.Profile()
profiler.enable()
#run backtest

evb_backtest.run()
profiler.disable()
stream = io.StringIO()
stats = pstats.Stats(profiler, stream=stream).sort_stats('cumulative')

        Date  TSLA  AAPL  MSFT  AVGO  AMZN  NVDA  JNJ  HD  BA  WMT  INTC  \
0 2023-07-05     1     1     1     1     1     1    0   0   0    0     0   

   QCOM  SBUX  AMD  MU  DIS  BAC  GOOG  
0     0     0    0   0    0    0     0  
Processing event: MARKET
Processing event: SIGNAL
Processing event: SIGNAL
Processing event: SIGNAL
Processing event: SIGNAL
Processing event: SIGNAL
Processing event: SIGNAL
Processing event: ORDER
Processing event: ORDER
Processing event: ORDER
Processing event: ORDER
Processing event: ORDER
Processing event: FILL
Processing event: FILL
Processing event: FILL
Processing event: FILL
Processing event: FILL
Event queue is empty, processed 17 event(s)
        Date  TSLA  AAPL  MSFT  AVGO  AMZN  NVDA  JNJ  HD  BA  WMT  INTC  \
1 2023-07-06     0     0     0     0     0     0    0   0   0    0     0   

   QCOM  SBUX  AMD  MU  DIS  BAC  GOOG  
1     0     0    0   0    0    0     0  
Processing event: MARKET
Event queue is empty, processed 1 event(s)
        

In [25]:
order_cache['2023-07-05']['TSLA']

{'result': 'SUCCESSFUL',
 'data': {'long': ['TSLA20240621C333.33'],
  'short': ['TSLA20240621C340'],
  'trade_id': '&L:TSLA20240621C333.33&S:TSLA20240621C340',
  'close': 1.5749999999999993}}

In [35]:
(evb_backtest.portfolio.options_data['TSLA20240621C333.33'] - evb_backtest.portfolio.options_data['TSLA20240621C340']).head(50)

,Open,High,Low,Close,Volume,Bid_size,CloseBid,Ask_size,CloseAsk,Midpoint,Weighted_midpoint
Datetime,,,,,,,,,,,
2023-07-05,2.40,2.40,2.75,2.37,21,69,1.95,8,2.05,2.000,1.935204
2023-07-05,2.40,2.40,2.75,2.37,21,69,1.95,8,2.05,2.000,1.935204
2023-07-06,1.11,0.73,2.74,1.26,-28,-3,1.90,19,1.95,1.925,1.945209
2023-07-06,1.11,0.73,2.74,1.26,-28,-3,1.90,19,1.95,1.925,1.945209
2023-07-07,0.80,0.80,3.20,2.80,-65,156,1.60,-166,1.60,1.600,1.229533
2023-07-07,0.80,0.80,3.20,2.80,-65,156,1.60,-166,1.60,1.600,1.229533
2023-07-10,1.80,0.98,2.17,1.70,8,88,1.40,-38,1.75,1.575,1.415262
2023-07-10,1.80,0.98,2.17,1.70,8,88,1.40,-38,1.75,1.575,1.415262
2023-07-11,1.50,0.60,2.40,1.30,-33,32,1.75,64,1.90,1.825,1.852402


In [13]:
pd.set_option('display.max_columns', 100)

In [14]:
eq = pd.DataFrame(evb_backtest.portfolio.weighted_holdings).set_index('datetime').copy()

In [26]:
evb_backtest.portfolio.plot_portfolio()

In [27]:
evb_backtest.portfolio.aggregate()

Start                                                    2023-07-05 00:00:00
End                                                      2024-07-01 00:00:00
Duration                                                   362 days 00:00:00
Exposure Time [%]                                                      100.0
Equity Final [$]                                                     38856.5
Equity Peak [$]                                                 40594.973877
Return [%]                                                         94.282489
Buy & Hold Return [%]                                              20.506831
CAGR [%]                                                           95.354757
Volatility Ann. [%]                                                76.683863
Sharpe Ratio                                                        1.241244
Sortino Ratio                                                        1.99267
Skew                                                                1.586575

In [29]:
evb_backtest.portfolio.get_trades()

,Ticker,PnL,ReturnPct,EntryPrice,EntryCommission,EntrySlippage,EntryMarketValue,TotalEntryCost,AuxilaryEntryCost,ExitPrice,ExitCommission,ExitSlippage,ExitMarketValue,TotalExitCost,AuxilaryExitCost,Quantity,EntryTime,ExitTime,Duration,Positions
0,TSLA,-17218.689932,-65.019936,264.821701,715.0,-1.961288,2198.038712,3626.077423,-1428.038712,92.634802,715.0,1.482819,1733.982819,1735.465638,-1.482819,11,2023-07-05,2023-08-02,28,&L:TSLA20240621C333.33&S:TSLA20240621C340
1,AAPL,-12947.146348,-48.891868,264.811857,390.0,-1.128860,1198.871140,1977.742280,-778.871140,135.340393,390.0,2.042359,1202.042359,1204.084718,-2.042359,6,2023-07-05,2023-08-04,30,&L:AAPL20240621C230&S:AAPL20240621C240
2,MSFT,-17030.068631,-64.242929,265.088609,455.0,0.620262,1400.620262,2311.240525,-910.620262,94.787923,455.0,-1.484542,1118.515458,1117.030916,1.484542,7,2023-07-05,2023-08-09,35,&L:MSFT20240621C370&S:MSFT20240621C375
3,AMZN,-17417.800259,-65.692356,265.141962,65.0,0.141962,200.141962,330.283925,-130.141962,90.963960,65.0,-0.036040,155.963960,155.927920,0.036040,1,2023-07-05,2023-10-25,112,&L:AMZN20240621C165&S:AMZN20240621C175
4,NVDA,265144.090525,1040.315600,254.868898,585.0,-1.179918,1708.820082,2877.640164,-1168.820082,2906.309803,585.0,-10.711771,26741.788229,26731.076458,10.711771,9,2023-07-05,2024-07-01,362,&L:NVDA20240621C770&S:NVDA20240621C800
5,BA,-16044.206254,-64.125099,250.201661,65.0,0.201661,185.201661,315.403322,-130.201661,89.759598,65.0,-0.240402,154.759598,154.519196,0.240402,1,2023-08-10,2023-08-17,7,&L:BA20240621C300&S:BA20240621C310
6,WMT,NaN,NaN,NaN,0.0,-0.000000,0.000000,NaN,NaN,NaN,0.0,0.000000,0.000000,NaN,NaN,0,2023-08-10,2023-08-17,7,&L:WMT20240621C175&S:WMT20240621C180
7,TSLA,-13530.395534,-51.048281,265.050951,195.0,0.152852,600.152852,990.305704,-390.152852,129.746995,195.0,-0.759014,584.240986,583.481972,0.759014,3,2023-08-17,2023-10-20,64,&L:TSLA20240920C305&S:TSLA20240920C315
8,AAPL,-17922.971075,-68.085872,263.240679,195.0,-0.777962,594.722038,983.944075,-389.222038,84.010968,195.0,0.032905,447.032905,447.065811,-0.032905,3,2023-08-31,2023-09-07,7,&L:AAPL20240920C260&S:AAPL20240920C310
9,INTC,NaN,NaN,NaN,0.0,-0.000000,0.000000,NaN,NaN,NaN,0.0,0.000000,0.000000,NaN,NaN,0,2023-09-15,2023-09-20,5,&L:INTC20240621C40&S:INTC20240621C45


## Extend for  get_port_stats
- Buy & Hold
- Dates
- Trades
- _strategy in Aggregate
- The function

In [17]:
evb_backtest.trades.to_csv('/Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/output/profitable_trades_options.csv')

In [38]:
op_trades = evb_backtest.trades.copy()
duration = op_trades.Duration.unique()
op_trades[op_trades.Ticker=='QCOM']

,Ticker,PnL,EntryPrice,ExitPrice,ReturnPct,Quantity,EntryTime,ExitTime,Duration,Positions
10,QCOM,270.205981,185.344291,455.550271,145.785975,1,2023-11-13,2024-07-01,231,&L:QCOM20250117C135&S:QCOM20250117C140


In [37]:
(200.595187 - 157.929765) * 11

469.31964200000004

In [19]:
trades_[trades_.Duration.isin(duration)]

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,Ticker


In [20]:
t = {'p': {}}
t.setdefault('p', [])


{}

In [21]:
stats.print_stats(30)
print(stream.getvalue())
with open('bactest_data.txt', 'w') as f:
  stream.seek(0)
  f.write(stream.read())
  f.flush()

         158689244 function calls (158065791 primitive calls) in 1277.285 seconds

   Ordered by: cumulative time
   List reduced from 2860 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000 1277.289  638.644 /Users/chiemelienwanisobi/miniconda3/envs/openbb/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3541(run_code)
        2    0.000    0.000 1277.289  638.644 {built-in method builtins.exec}
        1    0.042    0.042 1277.288 1277.288 /Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/backtest.py:37(run)
       64    0.000    0.000 1050.625   16.416 /Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/portfolio.py:290(update_signal)
       64    0.008    0.000 1050.623   16.416 /Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/portfolio.py:246(generate_order)
    62/31    0.017    0.000 1050.551   33.889 /Users/chiemelienwanisobi/cloned_repos/QuantTools/tr

In [22]:
evb_backtest.trades

,Ticker,PnL,EntryPrice,ExitPrice,ReturnPct,Quantity,EntryTime,ExitTime,Duration,Positions
0,TSLA,-469.319634,200.595187,157.929765,-21.269414,11,2023-07-05,2023-08-02,28,&L:TSLA20240621C333.33&S:TSLA20240621C340
1,AAPL,1.565517,200.443306,200.704226,0.130171,6,2023-07-05,2023-08-04,30,&L:AAPL20240621C230&S:AAPL20240621C240
2,MSFT,-276.544545,200.353116,160.846753,-19.718367,7,2023-07-05,2023-08-09,35,&L:MSFT20240621C370&S:MSFT20240621C375
3,AMZN,-43.770957,200.305228,156.534271,-21.852129,1,2023-07-05,2023-10-25,112,&L:AMZN20240621C165&S:AMZN20240621C175
4,NVDA,25078.793707,190.955251,2977.487885,1459.259494,9,2023-07-05,2024-07-01,362,&L:NVDA20240621C770&S:NVDA20240621C800
5,BA,-30.225169,185.573604,155.348435,-16.287429,1,2023-08-10,2023-08-17,7,&L:BA20240621C300&S:BA20240621C310
6,WMT,0.000000,NaN,NaN,NaN,0,2023-08-10,2023-08-17,7,&L:WMT20240621C175&S:WMT20240621C180
7,TSLA,-43.298192,200.384529,195.573618,-2.400839,9,2023-08-17,2023-10-20,64,&L:TSLA20240920C305&S:TSLA20240920C315
8,AAPL,-298.098050,199.060910,149.377901,-24.958697,6,2023-08-31,2023-09-07,7,&L:AAPL20240920C260&S:AAPL20240920C310
9,INTC,0.000000,NaN,NaN,NaN,0,2023-09-15,2023-09-20,5,&L:INTC20240621C40&S:INTC20240621C45


In [23]:
trades_

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,Ticker
0,34.0,504.0,524.0,279.795877,253.010232,-910.711955,-0.095733,2023-07-05,2023-08-02,28 days,TSLA
1,28.0,504.0,526.0,192.240502,185.493749,-188.909083,-0.035095,2023-07-05,2023-08-04,30 days,AAPL
2,17.0,504.0,529.0,336.262811,322.011093,-242.279208,-0.042383,2023-07-05,2023-08-09,35 days,MSFT
3,48.0,504.0,536.0,87.043588,82.000000,-242.092217,-0.057943,2023-07-05,2023-08-18,44 days,AVGO
4,11.0,504.0,583.0,130.695846,122.257034,-92.826927,-0.064568,2023-07-05,2023-10-25,112 days,AMZN
5,178.0,504.0,753.0,42.282471,123.470001,14451.380412,1.920123,2023-07-05,2024-07-01,362 days,NVDA
6,1.0,522.0,522.0,170.283917,168.355807,-1.928110,-0.011323,2023-07-31,2023-07-31,0 days,JNJ
7,5.0,522.0,556.0,332.810769,318.700396,-70.551863,-0.042398,2023-07-31,2023-09-18,49 days,HD
8,5.0,530.0,535.0,239.284572,224.549052,-73.677602,-0.061582,2023-08-10,2023-08-17,7 days,BA
9,5.0,530.0,535.0,53.784257,52.084613,-8.498219,-0.031601,2023-08-10,2023-08-17,7 days,WMT


In [24]:
from copy import deepcopy
test_data  =deepcopy(evb_backtest.portfolio.options_data['AAPL20240621C200'] - evb_backtest.portfolio.options_data['AAPL20240621C210'])
test_data[(test_data.index == '2023-07-05') | (test_data.index == '2023-10-26')]

KeyError: 'AAPL20240621C200'

In [33]:
evb_backtest.portfolio.all_positions


[{'GOOGL': {},
  'AMD': {},
  'MSFT': {},
  'datetime': Timestamp('2023-07-24 00:00:00')},
 {'GOOGL': {'position': {'long': ['GOOGL20240621P115'],
    'short': ['GOOGL20240621P100'],
    'trade_id': '&L:GOOGL20240621P115&S:GOOGL20240621P100',
    'close': 5.0},
   'quantity': 66.0,
   'market_value': 33000.0},
  'AMD': {'position': {'long': ['AMD20240621P110'],
    'short': ['AMD20240621P100'],
    'trade_id': '&L:AMD20240621P110&S:AMD20240621P100',
    'close': 4.75},
   'quantity': 69.47368421052632,
   'market_value': 33000.0},
  'MSFT': {},
  'datetime': Timestamp('2023-07-24 00:00:00')},
 {'GOOGL': {},
  'AMD': {},
  'MSFT': {},
  'datetime': Timestamp('2023-07-25 00:00:00')},
 {'GOOGL': {'position': {'long': ['GOOGL20240621C135'],
    'short': ['GOOGL20240621C147.5'],
    'trade_id': '&L:GOOGL20240621C135&S:GOOGL20240621C147.5',
    'close': 4.85},
   'quantity': 68.04121145934306,
   'market_value': 32999.98755778139},
  'AMD': {'position': {'long': ['AMD20240621P110'],
    'sho

In [34]:
evb_backtest.portfolio.options_data['GOOGL20240621P115'] - evb_backtest.portfolio.options_data['GOOGL20240621P100']

,Open,High,Low,Close,Volume,Bid_size,CloseBid,Ask_size,CloseAsk,Midpoint,Weighted_midpoint
Datetime,,,,,,,,,,,
2023-07-24,4.16,4.13,4.05,4.07,-9,-134,5.85,38,4.15,5.000,5.381509
2023-07-24,4.16,4.13,4.05,4.07,-9,-134,5.85,38,4.15,5.000,5.381509
2023-07-25,4.25,4.25,3.93,3.91,-3198,2,3.30,27,4.35,3.825,3.857051
2023-07-25,4.25,4.25,3.93,3.91,-3198,2,3.30,27,4.35,3.825,3.857051
2023-07-26,2.88,3.12,2.88,3.12,-910,18,3.02,-191,3.05,3.035,2.938961
2023-07-26,2.88,3.12,2.88,3.12,-910,18,3.02,-191,3.05,3.035,2.938961
2023-07-27,2.88,2.85,2.70,2.85,-25,-23,3.07,14,3.10,3.085,3.114296
2023-07-27,2.88,2.85,2.70,2.85,-25,-23,3.07,14,3.10,3.085,3.114296
2023-07-28,2.66,2.69,2.65,2.64,174,62,1.10,6,2.77,1.935,1.839688


In [24]:
from trade.helpers.helper import generate_option_tick
import pandas as pd

tick = 'AAPL'
exp = '2024-03-12'
right = 'C'
strike = 145.0
option_tick = generate_option_tick(tick, right, exp, strike)
option_tick


'AAPL20240312000145C'

In [35]:
evb_backtest.portfolio.current_positions

{'GOOGL': {}, 'AMD': {}, 'MSFT': {}}

In [26]:
stats.print_stats(20)  # Show the top 20 functions by cumulative time
print(stream.getvalue())

         53676519 function calls (52386882 primitive calls) in 216.194 seconds

   Ordered by: cumulative time
   List reduced from 2937 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000  216.195  108.097 /Users/chiemelienwanisobi/miniconda3/envs/openbb/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3541(run_code)
        2    0.000    0.000  216.195  108.097 {built-in method builtins.exec}
        1    0.000    0.000  216.195  216.195 /var/folders/j0/80hkbygd4lb27h9mw76gqzpw0000gn/T/ipykernel_40825/2620833521.py:1(<module>)
        1    0.003    0.003  216.195  216.195 /Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/backtest.py:55(run)
       21    0.000    0.000  215.775   10.275 /Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/portfolio.py:270(update_signal)
       21    0.003    0.000  215.775   10.275 /Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriv

In [27]:
trades = evb_backtest.portfolio.get_trades()
print(trades.to_string())

KeyError: 'option'

In [80]:
#Get all holdings
holdings = evb_backtest.get_all_holdings()
print(holdings.to_string())

               AMD    MSFT   AMZN   GOOGL   AAPL      cash  commission     total
datetime                                                                        
2024-03-05     0.0     0.0    0.0     0.0    0.0  100000.0         0.0  100000.0
2024-03-05     0.0     0.0    0.0     0.0    0.0  100000.0         0.0  100000.0
2024-03-05  3680.0     0.0    0.0     0.0    0.0   96320.0         0.0  100000.0
2024-03-05  3680.0  1624.0    0.0     0.0    0.0   94696.0         0.0  100000.0
2024-03-05  3680.0  1624.0  416.0     0.0    0.0   94280.0         0.0  100000.0
2024-03-06  4596.0  1402.0    0.0     0.0    0.0   94280.0         0.0  100278.0
2024-03-07  4804.0  1770.0    0.0     0.0    0.0   94280.0         0.0  100854.0
2024-03-08  4630.0  1880.0    0.0     0.0    0.0   94280.0         0.0  100790.0
2024-03-11  3746.0     0.0  380.0     0.0    0.0   94280.0         0.0   98406.0
2024-03-12  3310.0  1990.0  378.0     0.0    0.0   94280.0         0.0   99958.0
2024-03-13  3178.0  2096.0  

In [36]:
positions = evb_backtest.get_all_positions()
print(positions.to_string())

                             AMD                   AAPL                   MSFT                   GOOGL
datetime                                                                                              
2024-03-04                  None                   None                   None                    None
2024-03-04                  None                   None                   None                    None
2024-03-05  AMD-20240816-270.0-C                   None  MSFT-20240816-480.0-C                    None
2024-03-06  AMD-20240816-270.0-C                   None  MSFT-20240816-480.0-C                    None
2024-03-07  AMD-20240816-270.0-C                   None  MSFT-20240816-480.0-C                    None
2024-03-08  AMD-20240816-270.0-C                   None  MSFT-20240816-480.0-C                    None
2024-03-11  AMD-20240816-270.0-C                   None  MSFT-20240816-480.0-C                    None
2024-03-12  AMD-20240816-270.0-C                   None  MSFT-20240816-48

In [64]:
evb_backtest.portfolio.all_positions

[{'AMD': {'quantity': 0.0, 'option': None},
  'MSFT': {'quantity': 0.0, 'option': None},
  'AMZN': {'quantity': 0.0, 'option': None},
  'GOOGL': {'quantity': 0.0, 'option': None},
  'AAPL': {'quantity': 0.0, 'option': None},
  'datetime': Timestamp('2024-03-04 00:00:00')},
 {'AMD': {'quantity': 0.0, 'option': None},
  'MSFT': {'quantity': 0.0, 'option': None},
  'AMZN': {'quantity': 0.0, 'option': None},
  'GOOGL': {'quantity': 0.0, 'option': None},
  'AAPL': {'quantity': 0.0, 'option': None},
  'datetime': Timestamp('2024-03-04 00:00:00')},
 {'AMD': {'quantity': 2,
   'option': 'AMD-20240920-240.0-C',
   'entry_price': 2185.0},
  'MSFT': {'quantity': 2,
   'option': 'MSFT-20250117-540.0-C',
   'entry_price': 778.0},
  'AMZN': {'quantity': 2,
   'option': 'AMZN-20250321-205.0-C',
   'entry_price': 0.0},
  'GOOGL': {'quantity': 0.0, 'option': None},
  'AAPL': {'quantity': 0.0, 'option': None},
  'datetime': Timestamp('2024-03-05 00:00:00')},
 {'AMD': {'quantity': 2,
   'option': 'AMD-20

In [7]:
from pprint import pprint

print('All positions:')
pprint(portfolio.all_positions)

print('\nCurrent holdings:')
pprint(portfolio.current_holdings)

print('\nCurrent positions:')
pprint(portfolio.current_positions)

All positions:
[{'AAPL': {'option': None, 'quantity': 0.0},
  'AMD': {'option': None, 'quantity': 0.0},
  'AMZN': {'option': None, 'quantity': 0.0},
  'GOOGL': {'option': None, 'quantity': 0.0},
  'MSFT': {'option': None, 'quantity': 0.0},
  'datetime': '20240226'},
 {'AAPL': {'option': None, 'quantity': 0.0},
  'AMD': {'option': None, 'quantity': 0.0},
  'AMZN': {'option': None, 'quantity': 0.0},
  'GOOGL': {'option': None, 'quantity': 0.0},
  'MSFT': {'option': None, 'quantity': 0.0},
  'datetime': Timestamp('2024-02-26 00:00:00')},
 {'AAPL': {'option': None, 'quantity': 0.0},
  'AMD': {'option': None, 'quantity': 0.0},
  'AMZN': {'option': None, 'quantity': 0.0},
  'GOOGL': {'option': None, 'quantity': 0.0},
  'MSFT': {'option': None, 'quantity': 0.0},
  'datetime': Timestamp('2024-02-26 00:00:00')}]

Current holdings:
{'AAPL': 0.0,
 'AMD': 0.0,
 'AMZN': 0.0,
 'GOOGL': 0.0,
 'MSFT': 0.0,
 'cash': 100000,
 'commission': 0.0,
 'total': 100000}

Current positions:
{'AAPL': {'option': N

# Test RiskManager 

In [13]:
from EventDriven.riskmanager import RiskManager
from dbase.DataAPI.ThetaData import list_contracts, retrieve_option_ohlc, is_theta_data_retrieval_successful #type: ignore
import datetime
import pandas as pd
import pandas_market_calendars as mcal
import unittest
import numpy as np
import pprint as pp

tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA', 'NFLX']


#generate date range 
nyse = mcal.get_calendar('NYSE')
year_ago_date = datetime.datetime.now() - datetime.timedelta(days=365)
schedule = nyse.schedule(start_date=year_ago_date, end_date=datetime.datetime.now())
date_range = mcal.date_range(schedule, frequency='1D')
dates = [date.strftime('%Y-%m-%d') for date in date_range]




class RiskManagerOperations(unittest.TestCase):
    def __init__(self):
        self.events = Queue(maxsize=0)  
        self.bars = HistoricTradeDataHandler(self.events, trades)
        self.risk_manager = RiskManager(bars=self.bars, events=self.events, initial_capital=1000000)
        
    def test_order_picker(self):
        ticker = np.random.choice(tickers)
        contract_date = np.random.choice(dates)
        contracts = list_contracts(ticker, pd.to_datetime(contract_date).strftime('%Y%m%d'))
        self.assertTrue(is_theta_data_retrieval_successful(contracts))
        
        contract = contracts.sample()
        print(contract)
        contract_right = contract['right'].values[0]
        contract_expiration = f"{contract['expiration'].values[0]}"
        contract_strike = float(contract['strike'].values[0])
        max_close = np.random.randint(1, 10)
        
        #order settings 
        moneyness_width = np.random.uniform(0.05, 0.1)
        rel_strike_long = np.random.uniform(1.05, 1.3) 
        rel_strike_short = np.random.uniform(0.7, 0.95)
        dte = np.random.randint(30, 365)
        
        order_settings = {
            'type': 'spread',
            'specifics': [
                {'direction': 'long', 'rel_strike': rel_strike_long, 'dte': dte, 'moneyness_width': moneyness_width},
                {'direction': 'short', 'rel_strike': rel_strike_short, 'dte': dte, 'moneyness_width': moneyness_width} 
            ],
            'name': 'vertical_spread'
        }
       
        try:
            self.order = self.risk_manager.OrderPicker.get_order(ticker, contract_expiration, contract_right, max_close, order_settings)
            self.assertIsInstance(self.order, dict)
            self.assertIsInstance(self.order['long'], list)
            self.assertIsInstance(self.order['short'], list)
            self.assertGreater(len(self.order['long']), 0)
            self.assertGreater(len(self.order['short']), 0)
            self.assertIsInstance(self.order['close'], float)
        except AssertionError as e:
            print(f"AssertionError: {e}")
            pp.pprint(contract)
            print(f"Ticker: {ticker}")
            print(f"Contract Date: {contract_date}")
            print(f"Contract Right: {contract_right}")
            print(f"Contract Expiration: {contract_expiration}")
            print(f"Contract Strike: {contract_strike}")
            print(f"Max Close: {max_close}")
            print(f"Order Settings: {order_settings}")
            raise
        except Exception as e:
            print(f"Exception: {e}")
            pp.pprint(contract)
            print(f"Ticker: {ticker}")
            print(f"Contract Date: {contract_date}")
            print(f"Contract Right: {contract_right}")
            print(f"Contract Expiration: {contract_expiration}, type: {type(contract_expiration)}")
            print(f"Contract Strike: {contract_strike}, type: {type(contract_strike)}")
            print(f"Max Close: {max_close}")
            print(f"Order Settings: {order_settings}")
            raise


ops = RiskManagerOperations()
ops.test_order_picker()

      root  expiration  strike right
3444  NVDA    20240301   270.0     C
Exception: Cannot set a DataFrame with multiple columns to the single column option_id
      root  expiration  strike right
3444  NVDA    20240301   270.0     C
Ticker: NVDA
Contract Date: 2024-02-09
Contract Right: C
Contract Expiration: 20240301, type: <class 'str'>
Contract Strike: 270.0, type: <class 'float'>
Max Close: 2
Order Settings: {'type': 'spread', 'specifics': [{'direction': 'long', 'rel_strike': 1.2305088940545148, 'dte': 284, 'moneyness_width': 0.08149699520032436}, {'direction': 'short', 'rel_strike': 0.9364378888363725, 'dte': 284, 'moneyness_width': 0.08149699520032436}], 'name': 'vertical_spread'}


ValueError: Cannot set a DataFrame with multiple columns to the single column option_id

In [15]:
from trade.assets.Stock import Stock
NVDA = Stock('NVDA', run_chain = False)

In [14]:
def generate_option_to_buy(underlier: Stock, contract_time):
        """
        Buy an option based on the underlier.
        """
        time = contract_time
        next_day_time = time + pd.DateOffset(days=1)
        print(time, next_day_time)
        option_spot = underlier.spot(ts=True, ts_start = time, ts_end = next_day_time)
        option_spot = option_spot.iloc[0] 
        stock_price = option_spot['open']#use open price as spot price on the assumption of making trades at start of day
        oom_benchmark = 0.1#10% out of the money 
        expiry_benchmark = time + pd.DateOffset(months=5)
        oom_price = stock_price * (1 + oom_benchmark)
        time_str = time.strftime("%Y%m%d")
        contracts = list_contracts(underlier.ticker, time_str)
        print(contracts)
        contracts = contracts[contracts['right'] == 'C'] 
        
        
        #Filter out contracts that are out of the money
        contracts = contracts[contracts['strike'] >= oom_price]
        
        print('comparing expiry')
        print(type(expiry_benchmark))
        print(type(contracts['expiration']))
        #filter out contracts that are  below the expiry benchmark
        contracts = contracts[pd.to_datetime(contracts['expiration'], format="%Y%m%d") >= expiry_benchmark]
        
        #select a random contract to buy
        contract = contracts.sample(n=1); 
        
        return contract
                

In [8]:
appl = Stock('AAPL')
c_time = pd.to_datetime('2024-05-06', format="%Y-%m-%d")
c_time_next = c_time + pd.DateOffset(days=1)
aapl_spot = appl.spot(ts=True, ts_start = c_time, ts_end = c_time_next)
aapl_spot.iloc[0]['open']


182.35000610351562

In [15]:
option = generate_option_to_buy(appl, c_time)
option

2024-05-06 00:00:00 2024-05-07 00:00:00
      root  expiration  strike right
0     AAPL    20241018   220.0     C
1     AAPL    20250919   215.0     C
2     AAPL    20241115   220.0     C
3     AAPL    20250321   220.0     C
4     AAPL    20250321   220.0     P
...    ...         ...     ...   ...
1060  AAPL    20240621   220.0     C
1061  AAPL    20240621   220.0     P
1062  AAPL    20240719   220.0     C
1063  AAPL    20250620   215.0     C
1064  AAPL    20240816   220.0     C

[1065 rows x 4 columns]
comparing expiry
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas.core.series.Series'>


,root,expiration,strike,right
456,AAPL,20250117,235.0,C
